# initialize

In [33]:
import cobra,json
import pandas as pd
import io, re, requests
from bs4 import BeautifulSoup
from copy import deepcopy

import sys,os
# keep going up until the "pycore" folder is found
wd = os.getcwd()
while not os.path.exists('pycore'):
	os.chdir('..')
base_path = os.getcwd()
sys.path.append(base_path)
from pycore import *

os.chdir(wd)

In [34]:
model_path = 'iRhtoC.json'
# model_path = '/Users/ejm6426/Documents/ioRBA/build_model/input/iIsor850.json'
old_model = load_cobra_model(model_path)
# sort metabolites and reactions by id
# old_model.metabolites = sorted(old_model.metabolites, key=lambda x: x.id)
# old_model.reactions = sorted(old_model.reactions, key=lambda x: x.id)
# save the model
# save_cobra_model(old_model, model_path)

In [35]:
protein_id_in_gene_format = 'annotation["uniprot"]' # write as if accessing "model.genes.<gene_id>." followed by the attribute
protein_id_in_gene_format = "notes['protein_id']" # write as if accessing "model.genes.<gene_id>." followed by the attribute
def get_protein_id(gene):
	"""
	Get the protein ID from a COBRApy gene object.
	"""
	# Check if the gene object has the attribute
	try:
		# If the attribute exists, return its value
		protein_id = eval(f"gene.{protein_id_in_gene_format}")  # get the protein ID from the gene object
		return protein_id
	except (KeyError, AttributeError) as e:
		# If the attribute does not exist, return None or handle it as needed
		# print(f"Attribute '{protein_id_in_gene_format}' not found in gene {gene.id}.")
		# You can also raise an exception or return a default value if needed
		# raise AttributeError(f"Attribute '{protein_id_in_gene_format}' not found in gene object.")
		return None
	# if hasattr(gene, protein_id_in_gene_format):
	# 	# If the attribute exists, return its value
	# 	return eval(f"gene.{protein_id_in_gene_format}")  # get the protein ID from the gene object
	# else:
	# 	# If the attribute does not exist, return None or handle it as needed
	# 	print(f"Attribute '{protein_id_in_gene_format}' not found in gene {gene.id}.")
	# 	# You can also raise an exception or return a default value if needed
	# 	# raise AttributeError(f"Attribute '{protein_id_in_gene_format}' not found in gene object.")

genes_without_protein_id = []
gene_protein_mapping = {}
for gene in old_model.genes:
	gene_protein_mapping[gene.id] = [get_protein_id(gene)]
	# if the gene doesn't have the attribute, print a warning
	if gene_protein_mapping[gene.id] in [[None],[]]:
		genes_without_protein_id.append(gene.id)
		# print(f"{gene.id} does not have the attribute '{protein_id_in_gene_format}'")
if genes_without_protein_id:
	print(f"Warning: {len(genes_without_protein_id)} genes do not have the attribute '{protein_id_in_gene_format}': {genes_without_protein_id}")

In [36]:
# regex = {'compartment':re.compile(r'^.+_(.+)$') # last underscore denotes compartment
		#  }
regex = {'compartment':re.compile(r'^(.+)_(.+)$') # last underscore denotes compartment
		 }

In [37]:
def extract_details_from_rxnid_SM(met):
	if met.compartment:
		regex_match = regex['compartment'].match(met.id)
		if regex_match.group(2) == met.compartment:
			return regex_match.group(1)
extract_details_from_rxnid_SM(old_model.metabolites.o2_e)
# make the new method available to cobra.metabolite.Metabolite
# cobra.core.metabolite.Metabolite.base_id = str(extract_details_from_rxnid_SM)
# cobra.core.metabolite.Metabolite.id_without_compartment = id_without_compartment
# type(old_model.metabolites.o2_e)

'o2'

In [38]:
def parse_reaction(reaction):
	# use regex['compartment'] to parse the reaction
	m = regex['compartment'].match(reaction)
	if m:
		# print the match
		print(m.group(1))
	else:
		print("No match found")
parse_reaction('xt_c_c0')

xt_c


In [39]:
# x = 'y{1+1}'
# eval(f'f"{x}"'), x
rxn = old_model.reactions.EX_o2_e
rxn = old_model.reactions.O2t_c_e
def rxn_id_format(rxn):
	# get the reaction name
	rxn_name = rxn.name
	# get the compartment
	m = regex['compartment'].match(rxn.id)
	if m:
		compartment = m.group(1)
		# replace the compartment with the compartment name
		rxn_name = rxn_name.replace(f'_{compartment}','')
	# return the formatted reaction name
	return rxn_name
rxn.compartments
# x

{'c', 'e'}

In [40]:
# create version of model condensing all compartments into one
def dict_from_metabolite(metabolite):
	"""Convert a cobra.Metabolite object to a dictionary, excluding private attributes."""
	import re
	metabolite_dict = {re.sub(r'^_(id|annotation)$', r'\1', key): value for key, value in metabolite.__dict__.items() if key not in ['_model', '_reaction','_bound']}
	return metabolite_dict
def unique_metabolites(cobra_model,diff_attr='version_diffs',match_logic="\met.formula and \met.charge and (\met.annotation['metanetx.chemical'] or \met.annotation['chebi'] or \met.annotation['pubchem.compound'] or \met.annotation['seed.compound'] or \met.annotation['biocyc'])"):
	"""Find all metabolites that represent unique compounds.

	**diff_attr**: str with name of attribute in model.metabolites that will store differences between metabolites deemed similar.
	
	**match_logic**: str with logical expression indicating how attributes will affect whether 2 metabolites are identical. 
		Write attributes prefaced by "\met" (representing "model.metabolites"), e.g. "\met.formula" or "\met.annotation['chebi']']"
		Operators include 'and', 'or', 'not'. Use parentheses to group expressions.

	
	Output: list with metabolites in JSON form.
	"""
	model = deepcopy(cobra_model)
	# obtain all types of attributes from the model's metabolites, to see if notation conflicts with match_logic format
	metabolite_attributes = set()
	for metabolite in model.metabolites:
		for key in metabolite.__dict__.keys():
			metabolite_attributes.add(key)
	logic_gates = [' or ', ' and ', ' not ']
	issues = []
	for metabolite_attribute in metabolite_attributes:
		for gate in logic_gates:
			if gate in metabolite_attribute:
				issues.append((metabolite_attribute, gate))
	# if any metabolite attributes contain logic words, raise an error
	if issues:
		raise ValueError(f"Attributes in match_logic must be in the format '\\met.attribute', but the following metabolite attributes overlap with logic gates in match_logic (either change the attributes or logic gate formatting): {issues}")
	
	# extract all variables from match_logic, starting with \met. and ending with the first logic gate; remove any closed parentheses at the end
	# split match_logic by logic gates to get all variables
	variables = set([re.sub(r'.*\\met.|\)$', '', v) for v in re.split(r'{}'.format('|'.join(logic_gates)), match_logic)])
	match_logic_simple = re.sub(r'\\met.', '', match_logic)
	
	unique_mets = [model.metabolites[0]] # start with 1st metabolite to streamline future comparisons
	for met in model.metabolites[1:]:
		is_unique = True # assume metabolite is unique until proven otherwise
		met2_index = 0
		for met2 in unique_mets:
			met2_index += 1
			# initialize all variables to None
			logic_values = {var: False for var in variables}
			# check if each variable (if present) is the same for both metabolites
			# print(met.id, met2)
			for var in variables:
				# only make logic_values[var] True if it's identical for both metabolites or if it's not present in one or both metabolites
				if hasattr(met, var) and hasattr(met2, var):
					if getattr(met, var, '') == getattr(met2, var, ''):
						# print(f'{var} is the same for {met.id} and {met2.id}')
						logic_values[var] = True
					# else:
					# 	print(f'{var} is different for {met.id} and {met2.id}: {getattr(met, var)} vs {getattr(met2, var)}')
				elif hasattr(met, var) or hasattr(met2, var):
					# print(f'{var} not present in {met.id} or {met2.id}')
					logic_values[var] = True
				else:
					# print(f'{var} is not present in {met.id} or {met2.id}')
					logic_values[var] = True
					pass
			# evaluate match_logic for met and met2
			expr = match_logic_simple
			for var in sorted(variables, key=len, reverse=True):
				expr = expr.replace(var, str(logic_values[var]))
			if eval(expr):
				print(f'{met.id} and {met2.id} are the same')
				is_unique = False
				break
		if is_unique:
			unique_mets.append(met)
		else:
			# find differences between metabolites
			diffs = {met.id: {}}
			for attr in dict_from_metabolite(met).keys():
				if attr not in ['id']: # filtering out ID since it's already used as key
					if getattr(met, attr, '') != getattr(met2, attr, ''):
						diffs[met.id][attr] = getattr(met, attr, '')
			# store differences in the diff_attr attribute within unique_mets
			# setattr(unique_mets[-1], diff_attr, diffs)
			# setattr(met2, diff_attr, diffs)

			# unique_mets[met2_index-1].setdefault(diff_attr, {}).update(diffs)
			# unique_mets[met2_index-1].version_diffs.setdefault(met.id, {}).update(diffs)
			if diff_attr not in unique_mets[met2_index-1].__dict__.keys():
				setattr(unique_mets[met2_index-1], diff_attr, {})
			unique_mets[met2_index-1].version_diffs.update(diffs)
			# setattr(unique_mets[met2_index-1], diff_attr, diffs)
	# remove all mets from model not in unique_mets
	model.metabolites = unique_mets
	return unique_mets, model

unique_mets, model_simplified = unique_metabolites(old_model)
# make dict version
unique_mets_dict = [dict_from_metabolite(met) for met in unique_mets]
mets_dict = [dict_from_metabolite(met) for met in old_model.metabolites]
# save to file
with open('unique_mets.json', 'w') as f:
	json.dump(unique_mets_dict, f, indent=2)
with open('mets.json', 'w') as f:
	json.dump(mets_dict, f, indent=2, sort_keys=True)

def make_model_with_single_compartment(model):
	"""Combine all metabolites and/or reactions in different compartments into one compartment, for easier updating and analysis.
	
	Output: JSON file with all compartments written as lists under "notes" as "all_compartments"
	"""
	
	model = deepcopy(model)
	if len(model.compartments) == 1:
		print('Model already has only one compartment')
		return model
	else:
		# combine all compartments into one
		compartment = 'c'
		for metabolite in model.metabolites:
			metabolite.compartment = compartment
		for reaction in model.reactions:
			reaction.compartment = compartment
		# for all mets with differences, make ["notes"]["all_compartments"] with lists of differences
		# how to deal w/ rxns involving multiple compartments? Make ["notes"]["all_compartments"] a dict with all compartment-specific names as keys and lists of compartment-specific names as values

	# find all metabolites 
	return model

10fthf_m and 10fthf_c are the same
12dag3p_m and 12dag3p_c are the same
13BDglucan_e and 13BDglucan_c are the same
13BDglucan_en and 13BDglucan_c are the same
14dmlanost_e and 14dmlanost_c are the same
16BDglucan_en and 13BDglucan_c are the same
1agp_rm and 1agp_l are the same
1agp_vm and 1agp_l are the same
1agpc_l and 1agpc_en are the same
1agpc_mm and 1agpc_en are the same
1agpc_rm and 1agpc_en are the same
1agpe_l and 1agpe_en are the same
1agpe_mm and 1agpe_en are the same
1agpe_rm and 1agpe_en are the same
1mag_m and 1mag_l are the same
1mag_vm and 1mag_l are the same
1p3h5c_m and 1p3h5c_c are the same
1pyr4h2c_x and 1p3h5c_c are the same
1pyr5c_m and 1pyr5c_c are the same
23dhmb_m and 23dhmb_c are the same
23dhmp_m and 23dhmp_c are the same
2dda7p_m and 2dda7p_c are the same
2dhp_c and 2ahbut_m are the same
2dhp_m and 2ahbut_m are the same
2mbac_e and 2mbac_c are the same
2mbald_e and 2mbald_c are the same
2mbald_m and 2mbald_c are the same
2mbtcoa_c and 2mbcoa_m are the same
2m

In [41]:
# sort metabolites by ID
model_simplified.metabolites = sorted(model_simplified.metabolites, key=lambda x: x.id)

In [42]:
def expand_metabolites(met_list,diff_attr='version_diffs'):
	"""Expand all metabolites in a list to include differences in attributes.
	
	Output: list with metabolites in JSON form.
	"""
	expanded_mets = []
	for met in met_list.copy():
		# if met has differences, expand it
		if hasattr(met, diff_attr):
			# get all differences
			diffs = getattr(met, diff_attr)
			# for each difference, create a new metabolite with that difference
			for new_met_id,diff in diffs.items():
				# print(new_met_id,diff)
				# create a new metabolite with the difference
				new_met = deepcopy(met)
				new_met.id = new_met_id
				for attr, value in diff.items():
					setattr(new_met, attr, value)
				expanded_mets.append(new_met)
				delattr(new_met, diff_attr)
		# remove the diff_attr attribute from the metabolite
		if hasattr(met, diff_attr):
			delattr(met, diff_attr)
		expanded_mets.append(met)
	return expanded_mets
expanded_mets = expand_metabolites(unique_mets)
# make dict version
expanded_mets_dict = [dict_from_metabolite(met) for met in expanded_mets]
# save to file
with open('expanded_mets.json', 'w') as f:
	json.dump(expanded_mets_dict, f, indent=2, sort_keys=True)

In [43]:
# find which mets aren't in the expanded list but are in old_model
expanded_met_ids = [met.id for met in expanded_mets]
mets_not_expanded = [met.id for met in old_model.metabolites if met.id not in expanded_met_ids]
mets_not_expanded
# make dict version
# mets_not_expanded_dict = [dict_from_metabolite(met) for met in mets_not_expanded]
# save to file
# with open('mets_not_expanded.json', 'w') as f:
# 	json.dump(mets_not_expanded_dict, f, indent=2)

[]

In [44]:
# with open('~/Downloads/bigg_mets.json', 'r') as f:
# 	bigg = json.load(f)

# with open('bigg_mets.json', 'w') as f:
# 	json.dump(bigg, f, indent=4)

# mapping metabolites to ChEBI IDs

In [45]:
# map IDs to CHEBI numbers, and search web for missing IDs (can take a few mins)
model = deepcopy(old_model)
met_chebi_map = {}
ids_found = set()
# check if any metabolites have a kegg annotation
# for met in model.metabolites.copy():
mets_to_check = [i for i in model.metabolites if i.name == 'Cl-']
for met in mets_to_check:
	base_id = met.id.rsplit('_',1)[0]
	if 'kegg.compound' in met.annotation:
		ids_found.add('kegg.compound')
	if 'chebi' in met.annotation:
		ids_found.add('chebi')
		num = met.annotation['chebi'].replace('CHEBI:','')
		if num != '':
			met_chebi_map.setdefault(met.annotation['chebi'].replace('CHEBI:',''),{'default':base_id})[met.compartment] = met.id
	else:
		# try to find the chebi number from the name
		with requests.get('http://bigg.ucsd.edu/api/v2/universal/metabolites/'+base_id) as response:
			# if the request was successful
			if response.status_code == 200:
				# get the json data
				data = response.json()
				if 'database_links' in data:
					if 'metanetx.chemical' not in met.annotation and 'MetaNetX (MNX) Chemical' in data['database_links']:
						met.annotation['metanetx.chemical'] = data['database_links']['MetaNetX (MNX) Chemical'][0]['id']
						ids_found.add('metanetx.chemical')
					if 'kegg.compound' not in met.annotation and 'KEGG Compound' in data['database_links']:
						met.annotation['kegg.compound'] = data['database_links']['KEGG Compound'][0]['id']
						ids_found.add('kegg.compound')
					if 'seed.compound' not in met.annotation and 'SEED Compound' in data['database_links']:
						met.annotation['seed.compound'] = data['database_links']['SEED Compound'][0]['id']
						ids_found.add('seed.compound')
					if 'biocyc' not in met.annotation and 'BioCyc' in data['database_links']:
						met.annotation['biocyc'] = data['database_links']['BioCyc'][0]['id']
						ids_found.add('biocyc')
					# if the metabolite has a chebi number
					if 'CHEBI' in data['database_links']:
						ids_found.add('chebi')
						num = data['database_links']['CHEBI'][0]['id'].replace('CHEBI:','')
						if num != '':
							# search ChEBI for the number
							with requests.get('http://www.ebi.ac.uk/webservices/chebi/2.0/test/getCompleteEntity?chebiId='+num) as chebi_response:
								# if the request was successful
								if chebi_response.status_code == 200:
									# get the xml data
									chebi_data = BeautifulSoup(chebi_response.text, 'xml')
									print(chebi_data)
									# find the chebiId tag
									id_tag = chebi_data.find('chebiId')
									# if the tag was found, add it to the map and model
									if id_tag:
										num_primary = id_tag.text.replace('CHEBI:','')
										met_chebi_map.setdefault(num_primary,{'default':base_id})[met.compartment] = met.id
										# add the annotation to the model
										met.annotation['chebi'] = 'CHEBI:'+num_primary
										print('Found ChEBI ID',num_primary,'for',base_id)
									else:
										print('ChEBI ID not found for',num)
								else:
									print('ChEBI request failed for',num)
							# met_chebi_map.setdefault(data['chebi'].replace('CHEBI:',''),{'default':base_id})[met.compartment] = met.id
					
save_cobra_model(model,model_path)

In [46]:
# chebi_data.find('chebiId').text.replace('CHEBI:','')

In [47]:
# with open('~/Downloads/bigg_mets.json', 'r') as f:
with open('met_chebi_map.json', 'r') as f:
	file = json.load(f)

with open('met_chebi_map.json','w') as f:
	json.dump(file,f,indent=4)

In [48]:
# add some default mappings in case they are not found in the model
met_chebi_map_defaults = {'24875':{'default':'feCation'},'60240':{'default':'metal2'}
						  }
# add the default mappings to the main dictionary if they are not already there
for key in met_chebi_map_defaults.copy():
	if key not in met_chebi_map:
		met_chebi_map[key] = met_chebi_map_defaults[key]
# save the dictionary to a file
with open('met_chebi_map.json','w') as f:
	json.dump(met_chebi_map,f)

# download proteome files

In [49]:
# get proteome data from uniprot, for creating a model
uniprot_url = 'https://rest.uniprot.org/uniprotkb/'

# all proteins to include in your model (could be an organism ID, proteome ID, etc.; check to ensure your query is correct)
query = 'proteome:UP000239560'
# query = 'proteome:UP000249293'

# cc_alternative_products for alternative splicing
fields = 'accession,reviewed,id,protein_name,gene_names,organism_name,annotation_score,ec,sequence,mass,cc_sequence_caution,ft_chain,cc_ptm,cc_cofactor,cc_subunit,organelle,cc_subcellular_location,go_id,cc_function,cc_activity_regulation,ph_dependence,cc_miscellaneous,cc_alternative_products'
url = f'{uniprot_url}stream?compressed=true&fields={fields}&format=tsv&query=%28{query}%29'
with requests.get(url, stream=True) as request:
    request.raise_for_status()
    with open('stream.gz', 'wb') as f:
        for chunk in request.iter_content(chunk_size=2**20):
            f.write(chunk)

ConnectionError: HTTPSConnectionPool(host='rest.uniprot.org', port=443): Max retries exceeded with url: /uniprotkb/stream?compressed=true&fields=accession,reviewed,id,protein_name,gene_names,organism_name,annotation_score,ec,sequence,mass,cc_sequence_caution,ft_chain,cc_ptm,cc_cofactor,cc_subunit,organelle,cc_subcellular_location,go_id,cc_function,cc_activity_regulation,ph_dependence,cc_miscellaneous,cc_alternative_products&format=tsv&query=%28proteome:UP000239560%29 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x16f801e20>: Failed to resolve 'rest.uniprot.org' ([Errno 8] nodename nor servname provided, or not known)"))

In [ ]:
# uncompress file
import gzip
import shutil
with gzip.open('stream.gz', 'rb') as f_in:
	with open('proteome.tsv', 'wb') as f_out:
		shutil.copyfileobj(f_in, f_out)
# read the file into a pandas DataFrame
df = pd.read_csv('proteome.tsv', sep='\t')

# sort the DataFrame by a specific column, e.g., 'Entry'
df_sorted = df.sort_values(by='Entry')

# save the sorted DataFrame back to the file
df_sorted.to_csv('proteome.tsv', sep='\t', index=False)

# draft protein stoichiometry sheet

In [ ]:
compartments_lowercase = {key: value.lower() for key, value in model.compartments.items()}

In [ ]:
def location_to_compartment(location):
	"""Convert a location string to a compartment string.
	
	**location**: str with the location of the protein, e.g. 'cytoplasm', 'mitochondria', etc.
	
	Output: str with the compartment name, e.g. 'c', 'm', etc.
	"""
	if pd.isna(location):
		return ''
	location = location.lower()
	# check if the location is in the values of compartments_lowercase
	compartment = next((k for k, v in compartments_lowercase.items() if v == location), location)
	return compartment

# create draft of protein stoich sheet from the file
df = pd.read_csv('proteome.tsv', sep='\t')
# make df_enz with columns 'enz', 'rxns', 'comments'
df_enz = pd.DataFrame(columns=['enz','rxns','comments'])
df['uniprot'] = df['Entry']
# get the protein ID from the gene_protein_mapping key with a value that contains the uniprot ID
for i in df.index:
	gene_src = [k for k, v in gene_protein_mapping.items() if df.loc[i,'uniprot'] in v]
	df.loc[i,'gene_src'] = gene_src[0] if gene_src else ''
	# sort the list by name and then by whether they have "notes" in their attributes (since in iIsor850, this is where lower e-value BLAST hits are denoted)
	if len(gene_src) > 1:
		df.loc[i,'gene_src'] = sorted(gene_src,key=lambda x: ('sc_bidirectional_blast' in model.genes.get_by_id(x).__dict__['notes'].keys(), x))[0]

# if any of the keys in gene_protein_mapping don't have a protein with them as their gene_src, print a warning
def get_protein_id_from_gene(gene_id,protein_ids):
	if protein_ids[0] is not None and gene_id not in df['gene_src'].values:
		for protein_id in protein_ids:
			if protein_id in df['uniprot'].values:
				print(f"Warning: {gene_id} has a protein ID in the proteome file ({protein_ids}) but its gene_src is {df.loc[df['uniprot'] == protein_id, 'gene_src'].values[0]}")
				return
		print(f"Warning: {gene_id} does not have a protein ID in the proteome file")
for gene_id, protein_ids in gene_protein_mapping.items():
	get_protein_id_from_gene(gene_id, protein_ids)

df['protein'] = df['Protein names']
# move 'Sequence' to 'sequence', and delete the original 'Sequence' column
df['sequence'] = df['Sequence']
df.drop(columns=['Sequence'], inplace=True)
# check "Subunit structure" for the lowest word ending in "mer"
def get_mer(x):
	try:
		# find the lowest value (monomer, dimer, trimer, etc.)
		for word,i in zip(['monomer','dimer','trimer','tetramer','pentamer','hexamer','heptamer','octamer','nonamer','decamer'],range(1,11)):
			# if the word is found, return the value
			if word in x.lower():
				# if i != 1:
				# 	print(i, x)
				return i
	except:
		pass
	return 1 # if no value is found, assume monomer
df_debug = pd.DataFrame()
chebi_ids_unknown = set()
for i in df.index:
	df.loc[i,'stoich'] = get_mer(df.loc[i,'Subunit structure'])
	df.loc[i, 'translation_loc'] = location_to_compartment(df.loc[i, 'Gene encoded by'])
	if pd.isna(df.loc[i,'Subcellular location [CC]']):
		df.loc[i,'subloc_assigned'] = ''
	else:
		# remove 'SUBCELLULAR LOCATION: ' and text between curly brackets for df.loc[i,'Subcellular location [CC]'], then split by ';', and convert into comma-separated list of compartments where possible
		df.loc[i,'subloc_assigned'] = ', '.join([location_to_compartment(loc.split('.')[0]) for loc in re.sub(r'; .*?(\.|$)','. ',re.sub(r'\s+', ' ', re.sub(r' \{.*?\}|\.$', '', df.loc[i,'Subcellular location [CC]'].replace('SUBCELLULAR LOCATION: ','')))).strip().split('. ') if loc != ''])
	# df.loc[i,'subloc_assigned'] = df.loc[i,'Subcellular location [CC]']
	df.loc[i,'comments'] = df.loc[i,'Cofactor']
	# check for alternative products
	if not pd.isna(df.loc[i,'Alternative products (isoforms)']) and df.loc[i,'Alternative products (isoforms)'] != 'ALTERNATIVE PRODUCTS:  ':
		df_debug.loc[i, ['Entry', 'Alternative products (isoforms)']] = df.loc[i, ['Entry', 'Alternative products (isoforms)']]
	ptm = str(df.loc[i,'Post-translational modification'])
	ptm_stoich = {}
	# Standard cleavage stoichiometries, in case explicitly cleaving the protein isn't necessary
	hamap_rule = set(re.findall(r'HAMAP-Rule:(\w+)',ptm))
	if hamap_rule:
		df_debug.loc[i, ['Entry', 'Post-translational modification']] = df.loc[i, ['Entry', 'Post-translational modification']]
		if hamap_rule.intersection(['MF_03209','MF_03208']): # Phosphatidylserine decarboxylase (https://doi.org/10.1016/S0005-2760(97)00101-X)
			ptm_stoich['nh3'] = ptm_stoich.get('nh3',0) + 1
			ptm_stoich['h2o'] = ptm_stoich.get('h2o',0) - 1
		if hamap_rule.intersection(['MF_03124']):
			ptm_stoich['h2o'] = ptm_stoich.get('h2o',0) - 1
	# print(re.findall(r'CHEBI:(\d+)',str(df.loc[i,'Cofactor'])))
	for num in re.findall(r'CHEBI:(\d+)', str(df.loc[i,'Cofactor'])):
		# get the default metabolite ID for the CHEBI number
		met_id = met_chebi_map.get(num,{}).get('default')
		if met_id:
			# add the stoichiometry to the dictionary
			ptm_stoich[met_id] = ptm_stoich.get(met_id,0) + 1
			df_debug.loc[i, ['Entry', 'Cofactor']] = df.loc[i, ['Entry', 'Cofactor']]
		else:
			chebi_ids_unknown.add(num)
		
	# convert the stoichiometries to a string, joining the keys and values with colons (and removing unnecessary spaces)
	df.loc[i,'PTM_stoich_edits'] = ','.join([f'{k}:{v}' for k,v in ptm_stoich.items()])
	if ptm_stoich:
		df_debug.loc[i, ['Entry', 'PTM_stoich_edits']] = df.loc[i, ['Entry', 'PTM_stoich_edits']]
	
	df.loc[i,'MW (g/mmol)'] = df.loc[i,'Mass'] / 1000 # predictions at roughly neutral pH
	df.loc[i,'manually reviewed'] = ''

for id in chebi_ids_unknown:
	print(f'CHEBI:{id} not found in model')
	# http://bigg.ucsd.edu/api/v2/universal/metabolites

# output the draft
df.to_csv('protein_stoich.tsv', index=False, sep='\t')

/var/folders/fx/q_x2rm651s91fj2m6_tb19pxvpxbmb/T/ipykernel_31669/3741532278.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'COFACTOR: Name=Mn(2+); Xref=ChEBI:CHEBI:29035; Evidence={ECO:0000256|ARBA:ARBA00001936};' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i,'comments'] = df.loc[i,'Cofactor']
/var/folders/fx/q_x2rm651s91fj2m6_tb19pxvpxbmb/T/ipykernel_31669/3741532278.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'A0A061B3S8' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_debug.loc[i, ['Entry', 'Post-translational modification']] = df.loc[i, ['Entry', 'Post-translational modification']]
/var/folders/fx/q_x2rm651s91fj2m6_tb19pxvpxbmb/T/ipykernel_31669/3741532278.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated 

CHEBI:58937 not found in model
CHEBI:38290 not found in model
CHEBI:18420 not found in model
CHEBI:61717 not found in model
CHEBI:58130 not found in model
CHEBI:15636 not found in model
CHEBI:29108 not found in model
CHEBI:15361 not found in model
CHEBI:48828 not found in model
CHEBI:29034 not found in model
CHEBI:29103 not found in model
CHEBI:58210 not found in model
CHEBI:29035 not found in model
CHEBI:57586 not found in model
CHEBI:29033 not found in model
CHEBI:60052 not found in model
CHEBI:23378 not found in model
CHEBI:60342 not found in model
CHEBI:58349 not found in model
CHEBI:60344 not found in model
CHEBI:47942 not found in model
CHEBI:25516 not found in model
CHEBI:456215 not found in model
CHEBI:190135 not found in model
CHEBI:57540 not found in model
CHEBI:29036 not found in model
CHEBI:49786 not found in model
CHEBI:597326 not found in model
CHEBI:49883 not found in model
CHEBI:48775 not found in model
CHEBI:29105 not found in model
CHEBI:57618 not found in model
CHEBI

In [ ]:
# get CHEBI IDs from BiGG (if available)
new_chebi_dict = {}
if chebi_ids_unknown:
	for id in chebi_ids_unknown:
		match_found = False
		with requests.get('http://www.ebi.ac.uk/webservices/chebi/2.0/test/getCompleteEntity?chebiId='+id, stream=True) as request:
			request.raise_for_status()
			new_chebi_dict[id] = request.text
			# find all databaseLinks using BeautifulSoup
			soup = BeautifulSoup(request.text, 'xml')
			# find all databaseLinks
			for link in soup.find_all('DatabaseLinks'):
				# if type is 'KEGG COMPOUND', get the ID
				if 'kegg.compound' in ids_found:
					if link.find('type').text == 'KEGG COMPOUND accession':
						kegg_id = link.find('data').text
						# add CHEBI ID to all metabolites with the same KEGG ID
						for met in model.metabolites:
							if 'chebi' not in met.annotation:
								if 'kegg.compound' in met.annotation and met.annotation['kegg.compound'] == kegg_id:
									met.annotation['chebi'] = 'CHEBI:'+id
									match_found = True
		# if not match_found:
			# # search http://bigg.ucsd.edu/api/v2/universal/metabolites/ + default metabolite ID
			# with requests.get('http://bigg.ucsd.edu/api/v2/universal/metabolites/'+met_chebi_map[id]['default'], stream=True) as request:
			# 	request.raise_for_status()
			# 	# add CHEBI ID to all metabolites with the same default ID
			# for met in model.metabolites:
			# 	if 'chebi' not in met.annotation:
			# 		if met.id.rsplit('_',1)[0] == met_chebi_map[id]['default']:
			# 			met.annotation['chebi'] = 'CHEBI:'+id
			# 			match_found = True
			# print(re.findall(r'<databaseLinks>(.*?)</databaseLinks>',request.text))
# save updated model
save_cobra_model(model, model_path)

In [ ]:
#http://bigg.ucsd.edu/api/v2/universal/metabolites/
chebi_ids_unknown

{'15361',
 '15636',
 '18420',
 '190135',
 '21137',
 '23378',
 '25213',
 '25516',
 '29033',
 '29034',
 '29035',
 '29036',
 '29103',
 '29105',
 '29108',
 '30413',
 '38290',
 '456215',
 '47942',
 '48775',
 '48828',
 '49786',
 '49883',
 '57540',
 '57586',
 '57618',
 '57692',
 '58130',
 '58210',
 '58349',
 '58937',
 '597326',
 '60052',
 '60242',
 '60342',
 '60344',
 '61717',
 '71302',
 '83088'}

In [ ]:
new_chebi_dict

{'58937': '<?xml version=\'1.0\' encoding=\'UTF-8\'?><S:Envelope xmlns:S="http://schemas.xmlsoap.org/soap/envelope/"><S:Body><getCompleteEntityResponse xmlns="https://www.ebi.ac.uk/webservices/chebi"><return><chebiId>CHEBI:58937</chebiId><chebiAsciiName>thiamine(1+) diphosphate(3-)</chebiAsciiName><definition>Dianion of thiamine(1+) diphosphate arising from deprotonation of the three OH groups of the diphosphate.</definition><status>CHECKED</status><smiles>Cc1ncc(C[n+]2csc(CCOP([O-])(=O)OP([O-])([O-])=O)c2C)c(N)n1</smiles><inchi>InChI=1S/C12H18N4O7P2S/c1-8-11(3-4-22-25(20,21)23-24(17,18)19)26-7-16(8)6-10-5-14-9(2)15-12(10)13/h5,7H,3-4,6H2,1-2H3,(H4-,13,14,15,17,18,19,20,21)/p-2</inchi><inchiKey>AYEKOFBPNLCAJY-UHFFFAOYSA-L</inchiKey><charge>-2</charge><mass>422.29100</mass><monoisotopicMass>422.02259</monoisotopicMass><entityStar>3</entityStar><Synonyms><data>thiamin pyrophosphate</data><type>SYNONYM</type><source>ChEBI</source></Synonyms><Synonyms><data>thiamin pyrophosphate(2-)</data>

In [ ]:
df_debug

,Entry,Post-translational modification,Cofactor,PTM_stoich_edits
88,A0A061B3S8,PTM: Phosphorylation at Ser-175 and Ser-176 pr...,NaN,NaN
182,A0A0K3C4H9,NaN,COFACTOR: Name=a divalent metal cation; Xref=C...,metal2:1
884,A0A0K3CB17,NaN,COFACTOR: Name=a divalent metal cation; Xref=C...,metal2:1
890,A0A0K3CB33,NaN,COFACTOR: Name=a divalent metal cation; Xref=C...,metal2:1
1560,A0A0K3CFC3,NaN,COFACTOR: Name=a divalent metal cation; Xref=C...,metal2:1
2010,A0A0K3CI45,NaN,COFACTOR: Name=a divalent metal cation; Xref=C...,metal2:1
2014,A0A0K3CI56,NaN,COFACTOR: Name=Fe cation; Xref=ChEBI:CHEBI:248...,feCation:1
2130,A0A0K3CIW6,NaN,COFACTOR: Name=Fe cation; Xref=ChEBI:CHEBI:248...,feCation:1
2204,A0A0K3CJD8,PTM: Is synthesized initially as an inactive p...,NaN,"nh3:1,h2o:-1"
2368,A0A0K3CKD3,NaN,COFACTOR: Name=a divalent metal cation; Xref=C...,metal2:1


In [1]:
s = 'COFACTOR: Name=Mg(2+); Xref=ChEBI:CHEBI:18420; Evidence={ECO:0000256|RuleBase:RU003843}; Name=Mn(2+); Xref=ChEBI:CHEBI:29035; Evidence={ECO:0000256|RuleBase:RU003843}; Note=Binds 2 divalent metal cations per subunit. Magnesium or manganese. {ECO:0000256|RuleBase:RU003843};'
cofs = s.split(' Name=')[1:] # separate each cofactor, removing the first 'COFACTOR: ' part
cof_dict = {}
for cof in cofs:
	name = cof.split(';')[0]
	# cof_dict[cof.split(';')[0]] = {'chebi':cof.split('Xref=ChEBI:CHEBI:')[1].split(';')[0],'evidence':cof.split('Evidence=')[1].split(';')[0],'note':cof.split('Note=')[1].split(';')[0],'n':int(cof.split('Binds ')[1].split(' ')[0])}
	cof_dict[name] = {'chebi':cof.split('Xref=ChEBI:CHEBI:')[1].split(';')[0],'evidence':cof.split('Evidence=')[1].split(';')[0],'n':1}
	if 'Note=' in cof:
		cof_dict[name]['note'] = cof.split('Note=')[1].split(';')[0]
		# check if 'Binds ' is in the note
		if 'Binds ' in cof_dict[name]['note']:
			cof_dict[name]['n'] = int(cof_dict[name]['note'].split('Binds ')[1].split(' ')[0])
print(cof_dict)

{'Mg(2+)': {'chebi': '18420', 'evidence': '{ECO:0000256|RuleBase:RU003843}', 'n': 1}, 'Mn(2+)': {'chebi': '29035', 'evidence': '{ECO:0000256|RuleBase:RU003843}', 'n': 2, 'note': 'Binds 2 divalent metal cations per subunit. Magnesium or manganese. {ECO:0000256|RuleBase:RU003843}'}}


In [ ]:
df = pd.read_csv('protein_stoich.tsv', sep='\t')
# automatically determine gene_src by comparing uniprot IDs to PROTEIN_stoich_curation.tsv, if available
if os.path.exists('PROTEIN_stoich_curation.xlsx'):
    df_cur = read_spreadsheet('PROTEIN_stoich_curation.xlsx')
    # Drop duplicate uniprot entries to ensure unique index for mapping
    df_cur_unique = df_cur.drop_duplicates(subset='uniprot', keep='first')
    df['gene_src'] = df['uniprot'].map(df_cur_unique.set_index('uniprot')['id'])
# set gene_src to the index of df
df.index = df['gene_src']
df

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Annotation,EC number,Mass,Sequence caution,...,gene_src,protein,sequence,stoich,translation_loc,subloc_assigned,comments,PTM_stoich_edits,MW (g/mmol),manually reviewed
gene_src,,,,,,,,,,,,,,,,,,,,,
KU80,A0A059XCF6,unreviewed,A0A059XCF6_RHOTO,ATP-dependent DNA helicase II subunit 2 (EC 3....,KU80 FGENESH: predicted gene_8.185 AAT19DRAFT_...,Rhodotorula toruloides (Yeast) (Rhodosporidium...,3.0,3.6.4.12,102398,NaN,...,KU80,ATP-dependent DNA helicase II subunit 2 (EC 3....,MALASRTASLFLIDCSASMNRVRDFEVGYGEHKTVRRRSGIDVAKE...,1.0,NaN,"chromosome, telomere, n",NaN,NaN,102.398,NaN
RPL18A,A0A061AD89,unreviewed,A0A061AD89_RHOTO,RHTO0S01e01068g1_1 (Ribosomal protein L18e/L15P),AAT19DRAFT_9651 RHTO0S_01e01068g,Rhodotorula toruloides (Yeast) (Rhodosporidium...,1.0,NaN,21149,NaN,...,RPL18A,RHTO0S01e01068g1_1 (Ribosomal protein L18e/L15P),MGIDIERHHVKKGNRTAPKSEDPYLLLLVKLYRFLARRTDSRFNKV...,1.0,NaN,NaN,NaN,NaN,21.149,NaN
RT09658,A0A061ADB9,unreviewed,A0A061ADB9_RHOTO,Elongin-C,FGENESH: predicted gene_10.312 AAT19DRAFT_9658...,Rhodotorula toruloides (Yeast) (Rhodosporidium...,1.0,NaN,11547,NaN,...,RT09658,Elongin-C,MADADSWVTLVSSDGHRFILPRSAALGSEMIKNTLSADFVEAQTGV...,1.0,NaN,n,NaN,NaN,11.547,NaN
RT09347,A0A061ADZ3,unreviewed,A0A061ADZ3_RHOTO,Transcription initiation factor IIA subunit 2,FGENESH: predicted gene_10.39 AAT19DRAFT_9347 ...,Rhodotorula toruloides (Yeast) (Rhodosporidium...,2.0,NaN,13453,NaN,...,RT09347,Transcription initiation factor IIA subunit 2,MSGPAAPAQAATQTPFYQIYRRSALGTALVEALDELINSGHINPQL...,1.0,NaN,n,NaN,NaN,13.453,NaN
RPL19A,A0A061AEC3,unreviewed,A0A061AEC3_RHOTO,RHTO0S01e09934g1_1 (Ribosomal protein L19e-dom...,AAT19DRAFT_12418 RHTO0S_01e09934g,Rhodotorula toruloides (Yeast) (Rhodosporidium...,1.0,NaN,22470,NaN,...,RPL19A,RHTO0S01e09934g1_1 (Ribosomal protein L19e-dom...,MSNLRSQKRLAASVFGVGKRKIYLDPAHLGEIANANSRQNIRRLKK...,1.0,NaN,NaN,NaN,NaN,22.470,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,A0A2T0AJR7,unreviewed,A0A2T0AJR7_RHOTO,BRCA1-associated protein,AAT19DRAFT_9248,Rhodotorula toruloides (Yeast) (Rhodosporidium...,2.0,NaN,81613,NaN,...,NaN,BRCA1-associated protein,MPELLRRDSEDNRRYRVVIECYDHTSPRSINPLIPNTATRLPHTVD...,1.0,NaN,NaN,NaN,NaN,81.613,NaN
NaN,A0A2T0AJT4,unreviewed,A0A2T0AJT4_RHOTO,Uncharacterized protein,AAT19DRAFT_9258,Rhodotorula toruloides (Yeast) (Rhodosporidium...,1.0,NaN,28338,NaN,...,NaN,Uncharacterized protein,MPSVDTEPARPSLRKAIAQLPLPPVPAITSSTWRTHSSFHSAQSWA...,1.0,NaN,NaN,NaN,NaN,28.338,NaN
NaN,B3TQJ2,unreviewed,B3TQJ2_RHOTO,Rhodoturucine A,RHA2 AAT19DRAFT_10632 AAT19DRAFT_10637 AAT19DR...,Rhodotorula toruloides (Yeast) (Rhodosporidium...,1.0,NaN,6696,NaN,...,NaN,Rhodoturucine A,MAPITLAANQEVAAAAPALVNEEHNPTVTSYMACTVSRKDVSVEMV...,1.0,NaN,NaN,NaN,NaN,6.696,NaN


In [ ]:
#### Load data
import cobra,json,os,sys,openpyxl,re,pandas as pd
from collections import OrderedDict
from copy import deepcopy

model_root_path = '../../'
sys.path.append(model_root_path)
from pycore.gsm_custom_functions import *
from pycore.cobrapy_custom_extras import *

# Protein
df_pro = read_spreadsheet('PROTEIN_stoich_curation.xlsx')

# Enzyme
df_enz = read_spreadsheet('ENZYME_stoich_curation.tsv')

whole_biomass_rxns = ['BIOSYN-' + i for i in ['BIODILAERO', 'BIODILAERO-NOGAM','BIOXYL-PINHEIRO','BIOXYL-PINHEIRO-NOGAM']] + ['BIOMASS_xyl_hybrid']

# checks which rxns to ignore when seeing if all necessary enzymes are modeled. Ignore blocked rxns (to avoid unnecessary work from adding them)
rxns_not_needing_enzymes = list(dict.fromkeys([r.id for r in model.reactions if r.bounds == (0,0)] + [rxn.id for rxn in find_biomass_reactions(model)] + ['BIOMASS_MFA','BIOMASS_MFA_NO_GAM','BIOMASS_RBA']))
# Gene-Protein Reaction associations (GPRs) representing spontaneous reactions (i.e., no enzyme needed in model)
spont_GPRs = ['SPONT', 'UNKNOWN']
# take IDs of all rxns
print('Reactions to ignore when checking if enzymes must be added:')
print(rxns_not_needing_enzymes)
# Expanding df_pro to include sublocations as separate proteins
## remove the 'id' column
df_pro = df_pro.drop(columns=['id'])
# if the 'gene_src' column is empty, fill it with the 'name' column
df_pro['gene_src'] = df_pro['gene_src'].fillna(df_pro['name'])
## for each protein, if there are commas in subloc_assigned, split them into separate entries with 'id' modified to reflect sublocation
df_pro['subloc_assigned'] = df_pro['subloc_assigned'].apply(lambda x: x.split(',') if type(x) == str and ',' in x else x)
# add the 'id' column back in, assigning it to "gene_src" plus "subloc_assigned" if the latter is not 'unknown'
# modify IDs to reflect sublocation
# add location only if there are multiple proteins with the same gene source
df_pro = df_pro.explode('subloc_assigned').reset_index(drop=True)
if 'id' not in df_pro.columns:
	df_pro['id'] = df_pro.apply(
		lambda row: row['id_manual_override (clear when using new sheet)']
		if pd.notna(row['id_manual_override (clear when using new sheet)'])
		else (
			str(row['gene_src'])
			+ (
				'_' + row['subloc_assigned']
				if row['subloc_assigned'] != 'unknown'
				and df_pro[df_pro['gene_src'] == row['gene_src']].shape[0] > 1
				else ''
			)
		),
		axis=1
	)
df_pro.index = df_pro.id.to_list()
# calculate MW of enzymes = sum of MW of all proteins in protein_stoich (MW from df_pro) * number of protein_stoich

# if there are any rows in df_enz with 'dir' that's empty, create 2 new rows for each of those, one with 'dir' as 'FWD' and one with 'dir' as 'REV' (for faster writing to model)
if df_enz['dir'].isnull().any():
	print('Enzyme stoichiometry has empty "dir" values. Creating FWD and REV entries for those enzymes.')
	df_enz_fwd = df_enz.copy()
	df_enz_fwd['dir'] = df_enz_fwd['dir'].fillna('FWD')
	df_enz_rev = df_enz.copy()
	df_enz_rev['dir'] = df_enz_rev['dir'].fillna('REV')
	df_enz = pd.concat([df_enz_fwd[df_enz_fwd['dir'] == 'FWD'], df_enz_rev[df_enz_rev['dir'] == 'REV']], ignore_index=True)
	# sort df_enz by 'rxn_src' and 'dir'
	df_enz = df_enz.sort_values(by=['rxn_src', 'dir']).reset_index(drop=True)
	# Now df_enz_new contains all original rows, plus FWD and REV for those missing 'dir'

df_enz['id'] = df_enz.apply(lambda x: 'RXN-' + x['rxn_src'] + '_' + x['dir'] + '-' + x['enz'], axis=1)
df_enz.index = df_enz.id.to_list()
mw = 'MW (g/mmol)'
for enz in df_enz.index:
	enz_mw = 0
	for pro in df_enz.loc[enz, 'protein_stoich'].split(','):
		# find protein stoichiometry in df_pro
		delim = ':'
		if delim in pro:
			prot_id = pro.split(delim)[0].strip()
			prot_stoich = float(pro.split(delim)[1].strip())
			pro_matches = df_pro.loc[df_pro.gene_src == prot_id]
			# if no matches, check if prot_id is in df_pro.index
			if len(pro_matches) == 0:
				if prot_id in df_pro.index:
					pro_matches = df_pro.loc[prot_id]
				else:
					print(f'Warning: {prot_id} not found in protein stoichiometry file')
					continue
			pro_mw = pro_matches[mw]
			if isinstance(pro_matches[mw], pd.Series):
				if pro_matches[mw].nunique() > 1:
					print(f'Warning: {prot_id} has multiple MWs in df_pro')
				pro_mw = pro_matches[mw].iloc[0]
			if float(pro_mw) == 0:
				print(f'Warning: MW for {prot_id} is 0 in protein stoichiometry file')
				continue
			enz_mw += pro_mw * prot_stoich
	# set enz_mw to NaN if it is 0
	if enz_mw == 0:
		enz_mw = float('nan')
	df_enz.loc[enz, mw] = enz_mw
# move 'id' column to the front
# df_enz[['id'] + [col for col in df_enz.columns if col != 'id']].to_csv('./model/ENZYME_stoich_curation.tsv', sep='\t', index=False)
# list all genes, to map them to translation (PROSYN) rxns and other rxns making their products where needed
# combine all genes from df_pro and model
# combine all genes from df_pro and model, ensuring all are strings and not NaN
genes_model = [str(gene.id) for gene in model.genes if pd.notna(gene.id)]
genes_pro = [str(g) for g in df_pro.gene_src.to_list() if pd.notna(g)]
genes = sorted(list(dict.fromkeys(genes_model + genes_pro)))
gene_expression_dict = {}
#### Assemble reactions
import itertools
import cobra.manipulation

def find_true_combinations(expression, use_knockout=False, variables:list=[]):
    """
    Finds all combinations of boolean values that make the given expression true.

    Args:
        expression: The boolean expression as a string.
        use_knockout: If True, the function will test combinations where only one variable is False, and use those to eliminate redundant combinations, saving time. Will return an error if the expression contains 'not' operators unless set to 'ignore warning'.
        variables: A list of strings (variable names in expression) to consider. If none provided, the function will extract variable names from the expression. Added in case consistent RegEx parsing proves too unreliable.
    Returns:
        A list of dictionaries, where each dictionary represents a true combination.

    """

    # Extract variables from the expression
    if variables == []:
        variables = set(re.findall(r'\b\w+\b', expression))
        if use_knockout and any([var in ['not','!'] for var in variables]):
            raise ValueError("The expression contains 'not' operators, which may cause issues with the 'use_knockout' option. If you still want to use this option, set it to 'ignore warning'.")
        # remove 'and', 'or', 'not' from variables
        variables = [var for var in variables if var not in ['and', 'or', 'not']]
    # print(variables)

    # Generate all possible combinations of True/False values for variables
    combinations = itertools.product([True, False], repeat=len(variables))
    # sort combinations from fewest to most True values
    combinations = sorted(combinations, key=lambda x: sum(x))
    combos_to_check = combinations.copy()
    # print(combinations)
    # find all combinations where only one variable is False
    knockout_combos = []
    essential_vars = [] # variables that must be True
    # test if combo where all variables are True makes the expression True
    all_true = dict(zip(variables, [True]*len(variables)))
    expr = expression
    for var in sorted(variables, key=len, reverse=True):
        expr = expr.replace(var, str(all_true[var]))
    if use_knockout != False:
        if eval(expr):
            # for each variable, find the combination where only that variable is False
            for var in variables:
                combo = [True]*len(variables)
                combo[variables.index(var)] = False
                knockout_combos.append(combo)
                expr = expression
                for var2 in sorted(variables, key=len, reverse=True):
                    expr = expr.replace(var2, str(combo[variables.index(var2)]))
                # add variable to essential_vars if it must be True
                if not eval(expr):
                    essential_vars.append(var)
                    # remove all combinations where this variable is False from combos_to_check
                    combos_to_check = [combo for combo in combos_to_check if combo[variables.index(var)]]
    # if len(combinations) != len(combos_to_check):
    #     print(f'Reduced combinations to check from {len(combinations)} to {len(combos_to_check)} by eliminating redundant combinations.')

    true_combinations = []
    for combo in combos_to_check:
        # Create a dictionary mapping variables to their assigned values
        values = dict(zip(variables, combo))
        # print(values)
        # print(combo)
        # replace variables with their values in 'values'
        # cobra.manipulation.knock_out_model_genes

        expr = expression
        # Replace variables with their values in the expression, starting with the longest variable names to avoid the risk of replacing substrings
        for var in sorted(variables, key=len, reverse=True):
            expr = expr.replace(var, str(values[var]))

        # print(expression)
        # print(expr,type(expr))
        # Evaluate the expression
        if eval(expr):
            # check if the combination's True values are a subset of any previous ones; if so, don't add it
            # print('\t',values)
            redundant_combos = False
            true_vals = [var for var in values if values[var]]
            for prev in true_combinations:
                prev_vals = [var for var in prev if prev[var]]
                if set(true_vals).issubset(set(prev_vals)) or set(prev_vals).issubset(set(true_vals)):
                    redundant_combos = True
                    # print('redundant:',true_vals)
                    break
            if not redundant_combos:
                true_combinations.append(values)

    return true_combinations

# Automatically create placeholders for enzymes based on GPRs in model.reactions
rxns_to_skip = set(rxns_not_needing_enzymes)
enz_rxn_dict = {}
spont_rxn_dict = {}
rxn_updates = {rxn.id: {} for rxn in model.reactions}
GPR_enz_dict = {}
try:
	df_enz_rxns = read_spreadsheet('./input/ENZYME_stoich.tsv')
	# make set of rxns to skip, from combining the 'rxns' column cells in df_enz_rxns (to avoid reviewing existing rxns) and separating by commas. However, relying on this will prevent updates to their GPRs from being added to the model.
	# rxns_to_skip.update(set(df_enz_rxns.rxns.str.split(',').sum()))
	# make enz_rxn_dict from df_enz_rxns, with 'enz' as keys and 'rxns' and 'protein_stoich' as values
	for index, row in df_enz_rxns.iterrows():
		enz_rxn_dict[row['enz']] = {'rxns': sorted(list(dict.fromkeys(row['rxns'].split(',')))),
										'protein_stoich': {prot.split(':')[0]: float(prot.split(':')[1]) for prot in row['protein_stoich'].split(',')}}
		if not pd.isna(row[mw]):
			enz_rxn_dict[row['enz']][mw] = row[mw]
except:
	df_enz_rxns = pd.DataFrame(columns=['enz','rxns','protein_stoich',mw])
# add dict of protein_stoich to enz_rxn_dict, with each protein as a key and its stoichiometry as the value

# find all unique GPRs in model.reactions, and their associated reactions
for rxn in model.reactions:
	# print(rxn.id, rxn.gene_reaction_rule)
	if rxn.id in rxns_to_skip:
		continue
	if rxn.gene_reaction_rule not in spont_GPRs + ['']:
		if rxn.gene_reaction_rule in GPR_enz_dict.keys():
			# add rxn to corresponding enz in enz_rxn_dict
			enz_ids = GPR_enz_dict[rxn.gene_reaction_rule]
			for enz_id in enz_ids:
				enz_rxn_dict[enz_id]['rxns'].append(rxn.id)
				rxn_updates[rxn.id].setdefault('enz',enz_id)
		else:
			GPR_enz_dict[rxn.gene_reaction_rule] = set()
			enzs = sorted(find_true_combinations(rxn.gene_reaction_rule, use_knockout=True, variables=[g.id for g in rxn.genes]), key=lambda x: sorted(x.keys()))
			for enz in enzs:
				# print('\t',enz)
				enz_found = None
				# default: 1 copy of each protein is used
				# prot_stoich = ','.join([gene+':1' for gene in sorted(enz.keys()) if enz[gene]])
				# assemble prot_stoich as a dict with proteins as keys and their stoichiometry as values ('stoich' column of df)
				prot_stoich = {}
				for gene in sorted(enz.keys()):
					if enz[gene]:
						# if gene is in df, use its stoichiometry, otherwise use 1
						if gene in df.index:
							prot_stoich_val = df.loc[gene, 'stoich'] if 'stoich' in df.columns else 1
							# Remove unnecessary decimals: if value is integer-like, cast to int
							if isinstance(prot_stoich_val, float) and prot_stoich_val.is_integer():
								prot_stoich_val = int(prot_stoich_val)
							prot_stoich[gene] = prot_stoich_val
						else:
							# if gene is not in df, use 1 as default stoichiometry
							prot_stoich[gene] = 1

				# check if any enzymes with the same stoichiometry are already in enz_rxn_dict
				enz_id = ''.join(sorted(prot_stoich.keys()))

				for old_enz_id in enz_rxn_dict:
					# print(enz_id,enz_rxn_dict[enz_id])
					# automatically assumes enzymes are the same if they have the same proteins, regardless of order/count
					if sorted(enz_rxn_dict[old_enz_id]['protein_stoich'].keys()) == sorted(prot_stoich.keys()):
						enz_rxn_dict[old_enz_id]['rxns'].append(rxn.id)
						enz_found = old_enz_id
						break
				if not enz_found:
					# enz_rxn_dict[rxn.id] = enz
					enz_rxn_dict[enz_id] = {'rxns':[rxn.id],
											'protein_stoich':prot_stoich}
					rxn_updates[rxn.id].setdefault('enz',enz_id)
					GPR_enz_dict[rxn.gene_reaction_rule].add(enz_id)
				else:
					GPR_enz_dict[rxn.gene_reaction_rule].add(enz_found)
	else:
		# add to spont_rxn_dict under its GPR as a list of rxn IDs, if it is not already there
		spont_rxn_dict[rxn.gene_reaction_rule] = spont_rxn_dict.get(rxn.gene_reaction_rule, {'rxns':[],'protein_stoich':{'zeroCost':1}})
		spont_rxn_dict[rxn.gene_reaction_rule]['rxns'].append(rxn.id)
		rxn_updates[rxn.id].setdefault('gene_reaction_rule',rxn.gene_reaction_rule)
		# spont_rxn_dict[rxn.gene_reaction_rule] = spont_rxn_dict.get(rxn.gene_reaction_rule, [rxn.id])
		# spont_rxn_dict[rxn.gene_reaction_rule] = rxn.id
					
# give each enz in enz_rxn_dict a row in df_enz_rxns
new_rows = []
for enz in enz_rxn_dict:
	# only add enzymes that are not already in df_enz_rxns
	if df_enz_rxns[df_enz_rxns.enz == enz].empty:
		# calculate MW of each enzyme based on its protein stoichiometry
		if mw in enz_rxn_dict[enz] and enz_rxn_dict[enz][mw]:
			enz_mw = enz_rxn_dict[enz][mw]
		else:
			enz_mw = 0
			for prot in enz_rxn_dict[enz]['protein_stoich']:
				pro_mw = df_pro.loc[prot,mw] if prot in df_pro.index else 0
				# check in gene_src column
				if pro_mw == 0:
					if prot in df_pro.gene_src.to_list():
						# print(prot,df_pro.loc[df_pro.gene_src == prot,mw])
						# if there are multiple proteins with the same gene_src, check if their MW is the same
						# if not, print a warning
						# pick the first one
						pro_match = df_pro.loc[df_pro.gene_src == prot,mw].to_list()
						if df_pro.loc[df_pro.gene_src == prot,mw].nunique() > 1:
							print(f'Warning: {prot} has multiple MWs in df_pro')
						pro_mw = pro_match[0]
					else:
						print(f'Warning: {prot} not found in df_pro')
				enz_mw += pro_mw*enz_rxn_dict[enz]['protein_stoich'][prot]
		new_rows.append({'enz':enz,'rxns':','.join(enz_rxn_dict[enz]['rxns']),
						'protein_stoich':','.join([prot+':'+str(enz_rxn_dict[enz]['protein_stoich'][prot]) for prot in enz_rxn_dict[enz]['protein_stoich']]),
						mw:enz_mw,
						'status':'automatically added'})
df_enz_rxns = pd.concat([df_enz_rxns, pd.DataFrame(new_rows)], sort=False, ignore_index=True)
# sort by id, MW, and then by rxns
df_enz_rxns = df_enz_rxns.sort_values(by=['rxns',mw,'enz'])
# save automatically updated df_enz_rxns as separate file
df_enz_rxns.to_csv('../model/ENZYME_stoich.tsv', sep='\t', index=False)
# check if all rxns in model are accounted for in df_enz_rxns, and if they have been automatically updated
# make a list of all rxns in df_enz_rxns
enz_rxns = []
for index, row in df_enz_rxns.iterrows():
	enz_rxns += row['rxns'].split(',')
enz_rxns = sorted(list(dict.fromkeys(enz_rxns)))
# check if all rxns in model are in enz_rxns
missing_rxns = []
for rxn in model.reactions:
	if rxn.id not in rxns_not_needing_enzymes and rxn.id not in enz_rxns and rxn.id not in whole_biomass_rxns and rxn.gene_reaction_rule not in spont_rxn_dict.keys():
		missing_rxns.append(rxn.id)
if len(missing_rxns) > 0:
	print(f'Warning: the following reactions are not accounted for in df_enz_rxns:')
	for rxn in missing_rxns:
		print(rxn)
# remove rxns from rxn_updates if their value is {}
for rxn in rxn_updates.copy():
	if not rxn_updates[rxn]:
		del rxn_updates[rxn]
	else:
		print(f'Updating {rxn} with {rxn_updates[rxn]}')
# if rxn_updates isn't empty, print a warning
# if len(rxn_updates) > 0:
# 	raise ValueError(f'Warning: the following reactions have been automatically updated:\n{rxn_updates}')
# convert df_enz_rxns to old ENZYME_stoich_curation file format from 2024 version of scRBA, for compatibility
df_rxn_enz_pairs = pd.DataFrame(columns=['id','rxn_src','enz','protein_stoich'])
catalyst_rxn_dict = {**spont_rxn_dict, **enz_rxn_dict}
for k, v in catalyst_rxn_dict.items():
	if k in spont_rxn_dict:
		catalyst_rxn_dict[k]['rxns'].extend(spont_rxn_dict[k]['rxns']) # adds rxns that were overrided by merger with enz_rxn_dict
		catalyst_rxn_dict[k]['rxns'] = list(dict.fromkeys(catalyst_rxn_dict[k]['rxns']))
for enz in catalyst_rxn_dict:
	enz_id = enz if enz != '' else 'SPONT'
	# don't seem to need MW of enzyme in prior RBA code
	prot_stoich_str = 'zeroCost' if enz in spont_GPRs + [''] else ','.join([prot+':'+str(catalyst_rxn_dict[enz]['protein_stoich'][prot]) for prot in catalyst_rxn_dict[enz]['protein_stoich']])
	for rxn in catalyst_rxn_dict[enz]['rxns']:
		dirs = []
		# add FWD and REV to rxn_src if those directions are possible
		if model.reactions.get_by_id(rxn).lower_bound < 0:
			dirs.append('REV')
		if model.reactions.get_by_id(rxn).upper_bound > 0:
			dirs.append('FWD')
		for dir in dirs:
			# print(enz,rxn)
			# df_rxn_enz_pairs = df_rxn_enz_pairs.append({'id':f'RXN-{rxn}_{dir}-{enz}','enz':enz,'gpr':'','protein_stoich':','.join([prot+':'+str(enz_rxn_dict[enz]['protein_stoich'][prot]) for prot in enz_rxn_dict[enz]['protein_stoich']]),'status':'automatically added'}, ignore_index=True)
			df_rxn_enz_pairs.loc[len(df_rxn_enz_pairs)] = [f'RXN-{rxn}_{dir}-{enz_id}',rxn,enz,prot_stoich_str]
			# df_rxn_enz_pairs = pd.concat([df_rxn_enz_pairs,pd.DataFrame({'id':f'RXN-{rxn}_{dir}-{enz}','enz':enz,'protein_stoich':','.join([prot+':'+str(enz_rxn_dict[enz]['protein_stoich'][prot]) for prot in enz_rxn_dict[enz]['protein_stoich']])})], ignore_index=True)

# sort by ID
df_rxn_enz_pairs = df_rxn_enz_pairs.sort_values(by='id')
df_rxn_enz_pairs.to_csv('../model/ENZYME_rxn_pairs.tsv', sep='\t', index=False)

Reactions to ignore when checking if enzymes must be added:
['ABTD2Dy_c', 'ABTD4Dy_c', 'ABTLD_c', 'ACALDt_c_m', 'ACOAO100_x', 'ACOAO120_x', 'ACOAO40_x', 'ACOAO60_x', 'ACOAO80_x', 'ALCD2i2_c', 'ASNtps_v', 'ASPtps_v', 'BIOMASS_RBA', 'BIOMASS_MFA', 'BIOMASS_MFA_NO_GAM', 'BIOMASS_PinheiroEtAl2020', 'BIOMASS_xyl_hybrid', 'CTPS1_c', 'CYSItps_v', 'DDPA_m', 'EX_fald_e', 'FER_c', 'G3PD1i_m', 'GLNtps_v', 'GLUtps_v', 'GLYAT_c', 'HMGCOAS_m', 'ILEtps_v', 'LEUtps_v', 'NITRy_c', 'PHCHGS_m', 'PKETF_c', 'TYRtps_v', 'XU5PFGT_x', 'XYLI1_c', 'XYLI2_c', 'XYLK_c', 'XYLURx_c', 'XYLURy_c', 'BIOMASS']
Enzyme stoichiometry has empty "dir" values. Creating FWD and REV entries for those enzymes.
[{'rt012': False, 'rt04': True, 'rt0123': False, 'rt05': True, 'rt01': False}, {'rt012': True, 'rt04': False, 'rt0123': False, 'rt05': True, 'rt01': True}, {'rt012': False, 'rt04': False, 'rt0123': True, 'rt05': True, 'rt01': True}]
Updating 12AMANTF_g with {'enz': 'rt7201'}
Updating 13BDGLUCANt_c_en with {'gene_reaction_

/var/folders/fx/q_x2rm651s91fj2m6_tb19pxvpxbmb/T/ipykernel_31669/974584576.py:362: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_enz_rxns = pd.concat([df_enz_rxns, pd.DataFrame(new_rows)], sort=False, ignore_index=True)
